In [2]:
%matplotlib inline  
import scipy
import numpy as np
import pandas
from function_pO2 import calc_pO2

In [9]:
testdata = {'oxygen': [269.947, 270.084, 269.536, 269.307, 269.030, 269.358, 269.780, 269.129, 269.157, 269.275],
           'temp': [14.5144, 14.5175, 14.5139, 14.5135, 14.5082, 14.4960, 14.4788, 14.4703, 14.4414, 14.4498],
           'sal': [33.4526, 33.4527, 33.4529, 33.4527, 33.4525, 33.4529, 33.4527, 33.4521, 33.4545, 33.4535],
           'pres': [13.092, 14.100, 15.107, 16.114, 17.121, 18.128, 19.135, 20.142, 21.149, 22.157]}
           
testdata = pandas.DataFrame(testdata)
testdata[0:5]

,oxygen,pres,sal,temp
0,269.947,13.092,33.4526,14.5144
1,270.084,14.100,33.4527,14.5175
2,269.536,15.107,33.4529,14.5139
3,269.307,16.114,33.4527,14.5135
4,269.030,17.121,33.4525,14.5082


In [7]:
##UNITS
#O2 in umol/Kg
#Pressure in db
#Temp in Celsius

def calc_pO2(data):
#data = testdata   
    a_0 = 5.80871
    a_1 = 3.20291
    a_2 = 4.17887
    a_3 = 5.10006
    a_4 = -9.86643e-2
    a_5 = 3.80369
    b_0 = -7.01577e-3
    b_1 = -7.70028e-3
    b_2 =  -1.13864e-2
    b_3 = -9.51519e-3
    c_0 = -2.75915E-7

    data['tt'] = 298.15 - data['temp']
    data['tk'] = 273.15 + data['temp']
    data['ts'] = np.log(data['tt'] / data['tk'])

    #correct for pressure at depth
    V = 32e-6 #partial molar volume of O2 (m3/mol)
    R = 8.31 #Gas constant [J/mol/K]
    db2Pa = 1e4 #convert pressure: decibar to Pascal
    atm2Pa = 1.01325e5 #convert pressure: atm to Pascal

    #calculate pressure in dB from depth in m
    #Let zdepth = z[g=temp_rcp[d=2]]
    #let zgrid = temp_rcp[d=2]*0+zdepth
    #Let ztop = zgrid * (1.0076 + zgrid * (2.3487e-06 - zgrid * 1.2887e-11)) 

    #convert pressure from decibar to pascal
    data['dp'] = data['pres']*db2Pa
    data['pCor'] = np.exp((V*data['dp'])/(R*(data['temp']+273.15)))

    #Let o2_alpha = (o2_saturation / 0.21)  #0.21 is atm composition of O2
    #Let kh = o2_alpha*pCor
    #Let po2_raw = (o2_rcp[d=1]/kh)*101.32501   #convert po2 from atm to kPa  

    data['o2_sat'] = np.exp(a_0 + a_1*data['ts'] + a_2*data['ts']**2 + a_3*data['ts']**3 + a_4*data['ts']**4 + a_5*data['ts']**5 + data['sal']*(b_0 + b_1*data['ts'] + b_2*data['ts']**2 + b_3*data['ts']**3) + c_0*data['sal']**2)
    
    data['o2_alpha'] = (data['o2_sat'] / 0.21)  #0.21 is atmospheric composition of O2
    data['kh'] = data['o2_alpha']*data['pCor']
    data['po2'] = (data['oxygen'] / data['kh'])*101.32501  #convert po2 from atm to kPa
    #return data[['oxygen','pres','sal','temp','po2']]
    return data

In [8]:
hold = calc_pO2(testdata)
hold

,oxygen,pres,sal,temp,tt,tk,ts,dp,pCor,o2_sat,o2_alpha,kh,po2
0,50.000,13.092,33.4526,14.5144,283.6356,287.6644,-0.014104,130920.0,1.001754,252.877768,1204.179849,1206.292077,4.199854
1,270.084,14.100,33.4527,14.5175,283.6325,287.6675,-0.014126,141000.0,1.001889,252.862000,1204.104760,1206.379604,22.684621
2,269.536,15.107,33.4529,14.5139,283.6361,287.6639,-0.014101,151070.0,1.002024,252.879758,1204.189324,1206.626999,22.633952
3,269.307,16.114,33.4527,14.5135,283.6365,287.6635,-0.014098,161140.0,1.002159,252.882121,1204.200574,1206.800942,22.611463
4,269.030,17.121,33.4525,14.5082,283.6418,287.6582,-0.014061,171210.0,1.002295,252.909136,1204.329218,1207.092622,22.582747
5,269.358,18.128,33.4529,14.4960,283.6540,287.6460,-0.013975,181280.0,1.002430,252.969834,1204.618259,1207.545220,22.601805
6,269.780,19.135,33.4527,14.4788,283.6712,287.6288,-0.013855,191350.0,1.002565,253.056796,1205.032362,1208.123370,22.626382
7,269.129,20.142,33.4521,14.4703,283.6797,287.6203,-0.013795,201420.0,1.002700,253.100671,1205.241291,1208.495846,22.564826
8,269.157,21.149,33.4545,14.4414,283.7086,287.5914,-0.013593,211490.0,1.002836,253.242158,1205.915040,1209.334792,22.551518
9,269.275,22.157,33.4535,14.4498,283.7002,287.5998,-0.013652,221570.0,1.002971,253.201549,1205.721662,1209.303969,22.561980
